In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf

In [79]:
test = pd.read_csv(r'/Users/henry/Desktop/store-sales-time-series-forecasting/test.csv', parse_dates=["date"])
train = pd.read_csv(r'/Users/henry/Desktop/store-sales-time-series-forecasting/train.csv', parse_dates=["date"])
holidays_events = pd.read_csv(r'/Users/henry/Desktop/store-sales-time-series-forecasting/holidays_events.csv', parse_dates=["date"])
oil = pd.read_csv(r'/Users/henry/Desktop/store-sales-time-series-forecasting/oil.csv', parse_dates=["date"])
stores = pd.read_csv(r'/Users/henry/Desktop/store-sales-time-series-forecasting/stores.csv')
print(train.head())

   id       date  store_nbr      family  sales  onpromotion
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0
1   1 2013-01-01          1   BABY CARE    0.0            0
2   2 2013-01-01          1      BEAUTY    0.0            0
3   3 2013-01-01          1   BEVERAGES    0.0            0
4   4 2013-01-01          1       BOOKS    0.0            0


In [80]:
num_family = train.family.nunique()
num_store = train.store_nbr.nunique()
num_ts = train.groupby(["store_nbr", "family"]).ngroups
train_start = train.date.min().date()
train_end = train.date.max().date()
num_train_date = train.date.nunique()
train_len = (train_end - train_start).days + 1
test_start = test.date.min().date()
test_end = test.date.max().date()
num_test_date = test.date.nunique()
test_len = (test_end - test_start).days + 1
print("The number of product familes is " + str(num_family))
print("The number of stores is " + str(num_store))
print("The number of groups of stores and products is " + str(num_ts))
print("Train set starts on " + str(train_start))
print("Train set ends on " + str(train_end))
print("Train set includes " + str(num_train_date) + " days")
print("Train set lasts for " + str(train_len))
print("Test set starts on " + str(test_start))
print("Test set ends on " + str(test_end))
print("Test set includes " + str(num_test_date) + " days")
print("Test set lasts for " + str(test_len))

The number of product familes is 33
The number of stores is 54
The number of groups of stores and products is 1782
Train set starts on 2013-01-01
Train set ends on 2017-08-15
Train set includes 1684 days
Train set lasts for 1688
Test set starts on 2017-08-16
Test set ends on 2017-08-31
Test set includes 16 days
Test set lasts for 16


In [81]:
missing_dates = pd.date_range(train_start, train_end).difference(train.date.unique())
missing_dates = missing_dates.strftime("%Y-%m-%d").tolist()
print(missing_dates)

['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25']


In [82]:
#combine store info into train data on store number
stores_trimmed = stores[["store_nbr", "city", "state"]]
train.merge(stores_trimmed, on="store_nbr", how="left")

,id,date,store_nbr,family,sales,onpromotion,city,state
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha
...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha


In [83]:
#combine holiday info into train data
train['holidays'] = None
grouped = holidays_events.groupby('locale')
holidays = {locale: group for locale, group in grouped}
nationalholiday_map = holidays['National'].set_index('date')['description']
train['holidays'] = train['date'].map(nationalholiday_map)
print(train)

date
2012-08-10          Primer Grito de Independencia
2012-10-09             Independencia de Guayaquil
2012-10-12    Traslado Independencia de Guayaquil
2012-11-02                        Dia de Difuntos
2012-11-03                Independencia de Cuenca
                             ...                 
2017-12-22                              Navidad-3
2017-12-23                              Navidad-2
2017-12-24                              Navidad-1
2017-12-25                                Navidad
2017-12-26                              Navidad+1
Name: description, Length: 174, dtype: object


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [42]:
#fill missing values in training data
train[["sales", "onpromotion"]] = train[["sales", "onpromotion"]].fillna(0.)
train.id = train.id.interpolate(method="linear")

In [43]:
grouped = train.groupby(['store_nbr', 'family'])

# Convert to dictionary of DataFrames
dfs = {combination: group for combination, group in grouped}